# Project: Movie Recommender System

In [1]:
import os
os.getcwd()



'c:\\Users\\valar\\Documents\\Flatiron\\Assignments\\Phase 4\\Project\\Phase-4-Project---Movie-Recommendation-System\\notebook'

In [2]:
os.chdir(r'c:\Users\valar\Documents\Flatiron\Assignments\Phase 4\Project\Phase-4-Project---Movie-Recommendation-System')

In [3]:
%pwd


'c:\\Users\\valar\\Documents\\Flatiron\\Assignments\\Phase 4\\Project\\Phase-4-Project---Movie-Recommendation-System'

# EDA

In [4]:
# load the dataset
import pandas as pd
import numpy as np

df = pd.read_csv('./data/ratings.csv')
df.head()


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [6]:
# dropping the timestamp column as we don't need it
df = df.drop(['timestamp'],axis=1)

In [7]:
# checking that there are no null values in our dataset
df.isna().sum()

userId     0
movieId    0
rating     0
dtype: int64

In [8]:
mov_df = pd.read_csv('./data/movies.csv')
mov_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
tags_df = pd.read_csv('./data/tags.csv')
tags_df.shape

(3683, 4)

In [10]:
links_df = pd.read_csv('./data/links.csv')
links_df.shape

(9742, 3)

In [11]:
combined_df = pd.merge(df,mov_df,on= 'movieId')
combined_df.shape

(100836, 5)

In [12]:
combined_df.head()

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [13]:
# changing out df into the surprise 
# importing surprise
from surprise import Reader,Dataset
reader = Reader()
df_surprise = Dataset.load_from_df(df,reader)

### Doing neighbour-based similarity models
Looking at how many user's vs items we have. 

In [14]:
dataset = df_surprise.build_full_trainset()
print('Number of users',dataset.n_users,'\n')
print('Number of items',dataset.n_items)

Number of users 610 

Number of items 9724


In [15]:
# importing necessary libraries
from surprise.model_selection import cross_validate,GridSearchCV
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNBaseline,KNNWithMeans,KNNBasic


# Evaluating Model with SVD

In [16]:
# 
# param_grid = {'n_factors': [20,50,80],
#               'reg_all':[0.02,0.05],
#               'n_epochs':[15,20]}
# # Instantiate GridsearchCv 
# grid_svd = GridSearchCV(SVD,param_grid=param_grid,measures=['rmse'])

# # fitting to surprise dataset
# grid_svd.fit(df_surprise)


In [17]:
# print('Best Parameters:', grid_svd.best_params)
# print('Best Score', grid_svd.best_score)

Comparing our SVD model with KNNBasic,KNNBaseline,and KNNwithMeans

In [18]:

# cross-validating with KNNBasic
Basic = KNNBasic(sim_options={'name':'pearson','user_based':True})
cv_knn_basic = cross_validate(Basic,df_surprise,n_jobs=-1,measures=['rmse'])

In [19]:
Basic_trmse = np.mean(cv_knn_basic['test_rmse'])
print('Average test RMSE is:', Basic_trmse)

Average test RMSE is: 0.973456090061543


KNNBaseline

In [20]:
Baseline = KNNBaseline(sim_options={'name':"pearson",'user_based': True})
cv_knn_baseline = cross_validate(Baseline,df_surprise,n_jobs=-1,measures = ['rmse'])

Baseline_trmse = np.mean(cv_knn_baseline['test_rmse'])
print('Average test RMSE is:',Baseline_trmse )

Average test RMSE is: 0.8772424660880693


# KNNwithMeans

In [21]:
KwMeans = KNNWithMeans(sim_options={'name':"pearson",'user_based': True})
cv_knn_wMeans = cross_validate(KwMeans,df_surprise,n_jobs=-1,measures = ['rmse'])

kwMeans_trmse = np.mean(cv_knn_wMeans['test_rmse'])
print('Average test RMSE is:', kwMeans_trmse)

Average test RMSE is: 0.8957626144166999


In [22]:
# Summary = [{'Model': 'SVD', 'Best RMSE': grid_svd.best_score},
#            {'Model': 'KNNBasic', 'Best RMSE': Basic_trmse},
#            {'Model': 'KNNBaseline', 'Best RMSE': Baseline_trmse},
#            {'Model': 'KNNWithMeans', 'Best RMSE': kwMeans_trmse},]

# df_summary = pd.DataFrame(Summary)
# df_summary

# Making Movie Recommendations with SVD
SVD had the lowest rmse score at 0.86. We are going to fit and make predictions on this model.

In [23]:
# Instantiat an SVD model
# from GridSearch, Best Params: {'n_factors': 50, 'reg_all': 0.05, 'n_epochs': 20}}

svd = SVD(n_factors=50,reg_all=0.05,n_epochs=20)
svd.fit(dataset)

In [24]:
# Example prediction
svd.predict(1,3)

Prediction(uid=1, iid=3, r_ui=None, est=4.1081183071408, details={'was_impossible': False})

In [25]:
type(df_surprise)

surprise.dataset.DatasetAutoFolds

The estimated rating for user id 1, movieid 3 is 3.990 which is very close to its true rating = 4.0

In [26]:
# Created a helper function to get a rating list specific to one user
# def get_ratings_list(user_id):
#     if user_id not in df['userId'].values:
#         raise ValueError(f"userId not found in dataset!")
#     r_list = list(df.loc[df['userId'] == user_id, ['movieId','rating']].itertuples(index=False,name=None))
#     return r_list

In [27]:
# get_ratings_list()

In [65]:
# this function takes in a user_id, df_ratings(surprise dataset), and a number indicating how many 
# movie recommendations you want.
def movie_recs(user_id,df=df,num = 5):
    rating_list = []
    if user_id not in df['userId'].values:
         raise ValueError(f"userId not found in dataset!")
    for m_id in df['movieId'].unique():
        rating_list.append((m_id,svd.predict(user_id,m_id)[3]))
    
    ranked_list = sorted(rating_list,key=lambda x:x[1],reverse=True)[:num]
    # from sorted ranked list get movie_title using movie_id
    
    for i,val in enumerate(ranked_list):
        title = mov_df.loc[mov_df['movieId'] == val[0],'title'].iloc[0]
        print(f'Recommendation # {i + 1} : {title} predicted_rating : {val[1]} \n')
    
    
    
    
movie_recs(1,df,10)
    

Recommendation # 1 : Usual Suspects, The (1995) predicted_rating : 5 

Recommendation # 2 : Star Wars: Episode IV - A New Hope (1977) predicted_rating : 5 

Recommendation # 3 : Schindler's List (1993) predicted_rating : 5 

Recommendation # 4 : Star Wars: Episode V - The Empire Strikes Back (1980) predicted_rating : 5 

Recommendation # 5 : Apocalypse Now (1979) predicted_rating : 5 

Recommendation # 6 : Shawshank Redemption, The (1994) predicted_rating : 5 

Recommendation # 7 : Rear Window (1954) predicted_rating : 5 

Recommendation # 8 : Casablanca (1942) predicted_rating : 5 

Recommendation # 9 : Brazil (1985) predicted_rating : 5 

Recommendation # 10 : Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964) predicted_rating : 5 



In [66]:
movie_recs(3,df,4)

Recommendation # 1 : Hoop Dreams (1994) predicted_rating : 3.4913715373806893 

Recommendation # 2 : Road Warrior, The (Mad Max 2) (1981) predicted_rating : 3.4812465896827014 

Recommendation # 3 : Shawshank Redemption, The (1994) predicted_rating : 3.4401824978194173 

Recommendation # 4 : Legend of Drunken Master, The (Jui kuen II) (1994) predicted_rating : 3.3949163381864107 



# Hybrid Filtering
Solving the Cold Start Problem - by recommending the most popular movies by genre
2. Improving on collaborative filtering by considering user's genre preferences + svd

In [29]:
combined_df.head()

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [30]:
df_new = combined_df.copy()

In [31]:
df_new['genres'] = df_new['genres'].apply(lambda x: x.split('|'))

In [32]:
df_new['genres'].head()

0    [Adventure, Animation, Children, Comedy, Fantasy]
1    [Adventure, Animation, Children, Comedy, Fantasy]
2    [Adventure, Animation, Children, Comedy, Fantasy]
3    [Adventure, Animation, Children, Comedy, Fantasy]
4    [Adventure, Animation, Children, Comedy, Fantasy]
Name: genres, dtype: object

In [33]:
df_new['avg_rating'] = df_new.groupby('title')['rating'].transform('mean')
df_new['num_votes'] = df_new.groupby('title')['rating'].transform('count')
df_new.head()

,userId,movieId,rating,title,genres,avg_rating,num_votes
0,1,1,4.0,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",3.92093,215
1,5,1,4.0,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",3.92093,215
2,7,1,4.5,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",3.92093,215
3,15,1,2.5,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",3.92093,215
4,17,1,4.5,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",3.92093,215


In [34]:
# Explode the genre column
df_new = df_new.explode('genres')

In [35]:
df_new.shape

(274480, 7)

In [36]:
df_new.duplicated().sum()

0

In [53]:
df_new.head()

,userId,movieId,rating,title,genres,avg_rating,num_votes
0,1,1,4.0,Toy Story (1995),Adventure,3.92093,215
0,1,1,4.0,Toy Story (1995),Animation,3.92093,215
0,1,1,4.0,Toy Story (1995),Children,3.92093,215
0,1,1,4.0,Toy Story (1995),Comedy,3.92093,215
0,1,1,4.0,Toy Story (1995),Fantasy,3.92093,215


In [37]:
# Calculate average rating and num_votes() to get popularity metrics
#Computing weighted rating for each genre using IMDB'S weighted rating formula

C = df_new["avg_rating"].mean()
# getting only the top 10 most popular movies
m = df_new["num_votes"].quantile(0.90)

#filtering our dataframe and computing weighted_rating for top 10%  of movies by votes
top_10= df_new[df_new["num_votes"] >= m].copy()

top_10["weighted_rating"] = ((top_10["num_votes"]/(top_10["num_votes"] + m)) * top_10["avg_rating"] + (m/(top_10["num_votes"] + m)) * C)

# group top_10 df by genres and get movie count per genre
genre_table = (top_10.groupby(["genres"]).agg(avg_weighted_rating=("weighted_rating", "mean"),
                                num_movies=("title", "count"))
               .sort_values(by="num_movies", ascending=False))

genre_table = genre_table.reset_index()
genre_table

,genres,avg_weighted_rating,num_movies
0,Action,3.736997,3820
1,Adventure,3.745994,3655
2,Drama,3.874521,3464
3,Thriller,3.763501,3336
4,Comedy,3.709404,2217
5,Sci-Fi,3.780051,2113
6,Crime,3.874363,2044
7,Fantasy,3.725849,1113
8,Mystery,3.762117,1084
9,Romance,3.742155,1052


In [38]:
# get top 10 movies
top_10 = top_10.sort_values(by='weighted_rating',ascending=False).reset_index()
top_10 = top_10.drop_duplicates(subset='title').head(10)
top_10

,index,userId,movieId,rating,title,genres,avg_rating,num_votes,weighted_rating
0,16435,274,318,4.5,"Shawshank Redemption, The (1994)",Drama,4.429022,317,4.127200
634,1999,340,296,4.0,Pulp Fiction (1994),Thriller,4.197068,307,3.967226
1862,14425,317,2959,5.0,Fight Club (1999),Thriller,4.272936,218,3.956780
2734,1726,381,260,3.5,Star Wars: Episode IV - A New Hope (1977),Sci-Fi,4.231076,251,3.956600
3487,2495,122,356,5.0,Forrest Gump (1994),War,4.164134,329,3.955335
4803,12853,465,2571,4.0,"Matrix, The (1999)",Sci-Fi,4.192446,278,3.948949
5637,45548,610,858,5.0,"Godfather, The (1972)",Drama,4.289062,192,3.942099
6021,3761,539,527,3.5,Schindler's List (1993),War,4.225000,220,3.930485
6461,4514,446,593,5.0,"Silence of the Lambs, The (1991)",Crime,4.161290,279,3.929570
7298,652,249,50,4.0,"Usual Suspects, The (1995)",Mystery,4.237745,204,3.924633


In [43]:
# Created a helper function to get a rating list specific to one user
def get_ratings_list(user_id):
    if user_id not in df['userId'].values:
        r_list = []
    r_list = list(df.loc[df['userId'] == user_id, ['movieId','rating']].itertuples(index=False,name=None))
    return r_list

In [60]:
# improving movie_recs function
def movie_recommender(user_id,df=df,num = 5):
    rec_list = []
    ratings_list = get_ratings_list(user_id)
    # cold start problem
    if len(ratings_list) <= 5:
        print(f'Cold starter identified:Here are the top {num} most popular movies: \n' )
        display(top_10[['title','genres','rating']].head(num))
        return
  
    # get user_id's preferred genres
    pref_genres = df_new.loc[df_new['userId'] == user_id,"genres"].value_counts().index.to_list()
     
    for m_id in df['movieId'].unique():
        rec_list.append((m_id,svd.predict(user_id,m_id).est))
    
    # 
    rec_df = pd.DataFrame(rec_list,columns=['movieId','pred_rating'])
    rec_df = rec_df.merge(df_new[['movieId','title','genres']],on='movieId')
    # ranked_list = sorted(rec_list,key=lambda x:x[1],reverse=True)[:num]
    # from sorted ranked list get movie_title using movie_id
    
    final_recs = pd.DataFrame()
    
    for genre in pref_genres:
        g_df = rec_df[rec_df['genres'] == genre]
        g_df = g_df.sort_values('pred_rating', ascending=False)
        final_recs = pd.concat([g_df,final_recs])
    # since we exploded genres column, let us remove duplicates
    final_recs = final_recs.drop_duplicates(subset=['movieId']).reset_index(drop=True).head(num)
    
    for i, row in enumerate(final_recs.itertuples(), 1):
        print(
            f"Recommendation #{i}: {row.title} | "
            f"Genre: {row.genres} | "
            f"Predicted Rating: {row.pred_rating:.2f}"
        )

In [68]:
movie_recommender(0,df,4)

Cold starter identified:Here are the top 4 most popular movies: 



,title,genres,rating
0,"Shawshank Redemption, The (1994)",Drama,4.5
634,Pulp Fiction (1994),Thriller,4.0
1862,Fight Club (1999),Thriller,5.0
2734,Star Wars: Episode IV - A New Hope (1977),Sci-Fi,3.5
